Instruction Parser

In [1]:
import re
from typing import List, Dict

class InstructionParser:
    def __init__(self):
        self.patterns = [
            ("open", r"open|navigate to|go to"),
            ("click", r"click|press|tap"),
            ("fill", r"enter|type|fill"),
            ("assert", r"verify|check|assert")
        ]

    def parse(self, instructions: List[str]) -> List[Dict]:
        parsed_steps = []

        for step in instructions:
            step_lower = step.lower()
            action_found = False

            for action, pattern in self.patterns:
                if re.search(pattern, step_lower):
                    parsed_steps.append({
                        "action": action,
                        "raw_text": step
                    })
                    action_found = True
                    break

            if not action_found:
                parsed_steps.append({
                    "action": "unknown",
                    "raw_text": step
                })

        return parsed_steps


In [2]:
instructions = [
    "Open the login page",
    "Enter username as admin",
    "Click submit button"
]

parser = InstructionParser()
parsed = parser.parse(instructions)

for step in parsed:
    print(step)


{'action': 'open', 'raw_text': 'Open the login page'}
{'action': 'fill', 'raw_text': 'Enter username as admin'}
{'action': 'click', 'raw_text': 'Click submit button'}


Command Mapper

In [3]:
class CommandMapper:
    def map(self, parsed_steps):
        commands = []

        for step in parsed_steps:
            action = step["action"]

            if action == "open":
                commands.append({
                    "action": "open",
                    "url": "http://example.com"
                })

            elif action == "click":
                commands.append({
                    "action": "click",
                    "selector": "#submit"
                })

            elif action == "fill":
                commands.append({
                    "action": "fill",
                    "selector": "#input",
                    "value": "test_value"
                })

            else:
                commands.append({
                    "action": "noop",
                    "reason": "Unsupported instruction"
                })

        return commands


In [4]:
mapper = CommandMapper()
commands = mapper.map(parsed)

for cmd in commands:
    print(cmd)


{'action': 'open', 'url': 'http://example.com'}
{'action': 'fill', 'selector': '#input', 'value': 'test_value'}
{'action': 'click', 'selector': '#submit'}


Playwrite Code Generator

In [5]:
class PlaywrightCodeGenerator:
    def generate(self, commands):
        lines = []

        lines.append("from playwright.sync_api import sync_playwright")
        lines.append("")
        lines.append("with sync_playwright() as p:")
        lines.append("    browser = p.chromium.launch(headless=True)")
        lines.append("    page = browser.new_page()")

        for cmd in commands:
            if cmd["action"] == "open":
                lines.append(f'    page.goto("{cmd["url"]}")')

            elif cmd["action"] == "click":
                lines.append(f'    page.click("{cmd["selector"]}")')

            elif cmd["action"] == "fill":
                lines.append(
                    f'    page.fill("{cmd["selector"]}", "{cmd["value"]}")'
                )

        lines.append("    browser.close()")

        return "\n".join(lines)


In [6]:
generator = PlaywrightCodeGenerator()
code = generator.generate(commands)

print(code)


from playwright.sync_api import sync_playwright

with sync_playwright() as p:
    browser = p.chromium.launch(headless=True)
    page = browser.new_page()
    page.goto("http://example.com")
    page.fill("#input", "test_value")
    page.click("#submit")
    browser.close()


LangGraph Orchestrates

In [7]:
from typing import TypedDict, List
from langgraph.graph import StateGraph, END

class AgentState(TypedDict):
    instructions: List[str]
    parsed_steps: list
    commands: list
    generated_code: str


def parse_node(state: AgentState):
    state["parsed_steps"] = InstructionParser().parse(state["instructions"])
    return state


def map_node(state: AgentState):
    state["commands"] = CommandMapper().map(state["parsed_steps"])
    return state


def generate_node(state: AgentState):
    state["generated_code"] = PlaywrightCodeGenerator().generate(state["commands"])
    return state


graph = StateGraph(AgentState)
graph.add_node("parser", parse_node)
graph.add_node("mapper", map_node)
graph.add_node("generator", generate_node)

graph.set_entry_point("parser")
graph.add_edge("parser", "mapper")
graph.add_edge("mapper", "generator")
graph.add_edge("generator", END)

app = graph.compile()


In [8]:
result = app.invoke({
    "instructions": [
        "Open the login page",
        "Enter username as admin",
        "Click submit button"
    ]
})

print("PARSED STEPS:")
print(result["parsed_steps"])

print("\nCOMMANDS:")
print(result["commands"])

print("\nGENERATED CODE:")
print(result["generated_code"])


PARSED STEPS:
[{'action': 'open', 'raw_text': 'Open the login page'}, {'action': 'fill', 'raw_text': 'Enter username as admin'}, {'action': 'click', 'raw_text': 'Click submit button'}]

COMMANDS:
[{'action': 'open', 'url': 'http://example.com'}, {'action': 'fill', 'selector': '#input', 'value': 'test_value'}, {'action': 'click', 'selector': '#submit'}]

GENERATED CODE:
from playwright.sync_api import sync_playwright

with sync_playwright() as p:
    browser = p.chromium.launch(headless=True)
    page = browser.new_page()
    page.goto("http://example.com")
    page.fill("#input", "test_value")
    page.click("#submit")
    browser.close()
